In [1]:
import re
from functions import *
import langdetect

In [41]:
file = "files\\Men\\en_ 2022-2024 MAG CoP.pdf"
# file = "files/Men/Reglement-Bovenbouw-versie-1.1.pdf"
page = loadPages(file, 115)
text= page[0].get_text()
text


page range: [115]


'Code MAG July 2022\n111\n466.\n465.\nEG IV:  Sauts à partir de rondade - Round off entry vaults.- Saltos con entrada de rondada\n464.\n463.\n467. \nRondade, saut arrière avec 1/1 t. et renverse-\nment arrière et salto arrière tendu.\t\nScherbo entry to salto bwd. straight.\t\nEntrada de Scherbo mortal at ext.\n3.8\n(Scherbo)\n468. Scherbo avec ½ t.\t\nScherbo with ½ t.\nScherbo con ½ g.\n4.2\n470. Scherbo avec 3/2 t.\t\nScherbo with 3/2 t.\nScherbo con 3/2 g.\n5.0\n469. \nScherbo avec 1/1 t.\t\nScherbo with 1/1 t.\nScherbo con 1/1 g.\n4.6\n472. Scherbo avec 5/2 t.\t\nScherbo with 5/2 t.\nScherbo con 5/2 g.\n5.8\n471. \nScherbo avec 2/1 t.\t\nScherbo with 2/1 t.\nScherbo con 2/1 g.\n5.4\n(Shirai 3)\n461. \nRondade, saut arrière avec 1/1 t. et renverse-\nment arrière et salto arrière groupé avec 1/1 t.\t\nScherbo entry to salto bwd. tucked with 1/1\nturn.\t\nEntrada de Scherbo y mortal atrás agrupado\ncon 1/1 g.\n3.4\n462. \nRondade, saut arrière avec 1/1 t. et renverse-\nment arrière e

In [12]:

for line in text.split("\n"):
    print(line)

Code MAG July 2022
107
5.6
EG II:  Renversements avant avec ou sans tours simples et tout double salto avant.- Handspring salto vaults with or without simple twists, and all double salto fwd.- Palomas con o sin giros y dobles mortales ad.
5.2
248. 
247.
243
(Roche)
(Dragulescu)
225. 
Renversement avant et double salto avant gr.
	
Handspring fwd. and dbl. salto fwd. t.
 
Paloma doble mortal agrupado.
226. 
Roche avec ½ tour.
	
Roche with ½ turn.
 
Roche con ½ giro.
5.6
5.6
228. Renv. av. et salto av. gr. avec ½ t. et salto arr. gr.
	
Handspring fwd. and salto fwd. t. w. ½ t. and 
salto bwd. t.
 
Paloma y mortal ad con ½ g. y mortal at. agr.
6.0
(Zimmerman)
(Blanik)
(RI Se Gwang 2)
231. Renversement av. et double salto av. carpé.
	
Handspring fwd. and dbl. salto fwd. piked.
 
Paloma doble mortal ad. en carpa.
232. Dragulescu carpé.
	
Dragulescu piked.
 
Dragulescu en carpa.
235. 
236. 
237. 
233. 
238. 
227. 
234. 
244.
229. 
230. 
239. 
240. 
245. 
246. 
241. 
242. 



In [37]:
y = text.replace("\n", "|")

In [38]:
[y]

['Code MAG July 2022|111|466.|465.|EG IV:  Sauts à partir de rondade - Round off entry vaults.- Saltos con entrada de rondada|464.|463.|467. |Rondade, saut arrière avec 1/1 t. et renverse-|ment arrière et salto arrière tendu.\t|Scherbo entry to salto bwd. straight.\t|Entrada de Scherbo mortal at ext.|3.8|(Scherbo)|468. Scherbo avec ½ t.\t|Scherbo with ½ t.|Scherbo con ½ g.|4.2|470. Scherbo avec 3/2 t.\t|Scherbo with 3/2 t.|Scherbo con 3/2 g.|5.0|469. |Scherbo avec 1/1 t.\t|Scherbo with 1/1 t.|Scherbo con 1/1 g.|4.6|472. Scherbo avec 5/2 t.\t|Scherbo with 5/2 t.|Scherbo con 5/2 g.|5.8|471. |Scherbo avec 2/1 t.\t|Scherbo with 2/1 t.|Scherbo con 2/1 g.|5.4|(Shirai 3)|461. |Rondade, saut arrière avec 1/1 t. et renverse-|ment arrière et salto arrière groupé avec 1/1 t.\t|Scherbo entry to salto bwd. tucked with 1/1|turn.\t|Entrada de Scherbo y mortal atrás agrupado|con 1/1 g.|3.4|462. |Rondade, saut arrière avec 1/1 t. et renverse-|ment arrière et salto arrière groupé avec 3/2 t.\t|Scherbo e

In [42]:

pattern = r"(\d+)\.\s+(.*?)\s*(?=\|\d+\.|\Z)"
for res in re.findall(pattern, text.replace("\n", "|")):
    print(res)
    # if res[0] == "110":
    #     break

('467', '|Rondade, saut arrière avec 1/1 t. et renverse-|ment arrière et salto arrière tendu.\t|Scherbo entry to salto bwd. straight.\t|Entrada de Scherbo mortal at ext.')
('468', 'Scherbo avec ½ t.\t|Scherbo with ½ t.|Scherbo con ½ g.')
('470', 'Scherbo avec 3/2 t.\t|Scherbo with 3/2 t.|Scherbo con 3/2 g.')
('469', '|Scherbo avec 1/1 t.\t|Scherbo with 1/1 t.|Scherbo con 1/1 g.')
('472', 'Scherbo avec 5/2 t.\t|Scherbo with 5/2 t.|Scherbo con 5/2 g.')
('471', '|Scherbo avec 2/1 t.\t|Scherbo with 2/1 t.|Scherbo con 2/1 g.')
('461', '|Rondade, saut arrière avec 1/1 t. et renverse-|ment arrière et salto arrière groupé avec 1/1 t.\t|Scherbo entry to salto bwd. tucked with 1/1|turn.\t|Entrada de Scherbo y mortal atrás agrupado|con 1/1 g.')
('462', '|Rondade, saut arrière avec 1/1 t. et renverse-|ment arrière et salto arrière groupé avec 3/2 t.\t|Scherbo entry to salto bwd. tucked with 3/2|turn.\t|Entrada de Scherbo y mortal atrás agrupado|con 3/2 g.')
('457', '|Rondade, saut arrière avec 1/1

In [45]:
s = 'Rondade, ½ t. et renversement av. et double |salto avant groupé.\t|Round off, ½ t. a. hdspr. fwd. and double salto |fwd. tucked.|Rondada, ½ g. paloma doble mortal ad. agr.'

In [59]:
for x in re.split(r"\|(?=[A-Z])", s):
    print("t", x)

t Rondade, ½ t. et renversement av. et double |salto avant groupé.	
t Round off, ½ t. a. hdspr. fwd. and double salto |fwd. tucked.
t Rondada, ½ g. paloma doble mortal ad. agr.


In [14]:
for r in res.split("|"):
    print(r, langdetect.detect_langs(r) )



Appui   facial  horizontal.     (2      s.). [ro:0.9999938676465245]
 Support scale (2 s.). [fr:0.8571386350232229, it:0.1428585880455012]
     Apoyo  facial  horizontal      (2      s.). [es:0.9999951456983701]


In [34]:
loadConfig("element_grouping.yaml")["apparatuses"]["men"]["vault"]["type"]

Loading config file


['handstandoverslag / yamashita',
 'tsukahara',
 'arabier flik-flak / yurchenko',
 'streksprong']